In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
# Prepare training documents, which are labeled.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)
], ["id", "text", "label"])

In [0]:
# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
count_vectorizer = CountVectorizer(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, count_vectorizer, lr])

In [0]:
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for count_vectorizer.vocabSize and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(count_vectorizer.vocabSize, [1024, 16384, 262144]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

In [0]:
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
cross_val = CrossValidator(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(),
                           numFolds=2) # use 3+ folds in practice

In [0]:
# Run cross-validation, and choose the best set of parameters.
cvModel = cross_val.fit(training)

In [0]:
print("\nModel was fit using parameters: ")
print(cvModel.extractParamMap())

Model was fit using parameters: 
{Param(parent='CrossValidatorModel_698659934567', name='seed', doc='random seed.'): 7809051150349531440, Param(parent='CrossValidatorModel_698659934567', name='numFolds', doc='number of folds for cross validation'): 2, Param(parent='CrossValidatorModel_698659934567', name='foldCol', doc="Param for the column name of user specified fold number. Once this is specified, :py:class:`CrossValidator` won't do random k-fold split. Note that this column should be integer type with range [0, numFolds) and Spark will throw exception on out-of-range fold numbers."): '', Param(parent='CrossValidatorModel_698659934567', name='estimator', doc='estimator to be cross-validated'): Pipeline_4c1a9f170ba5, Param(parent='CrossValidatorModel_698659934567', name='estimatorParamMaps', doc='estimator param maps'): [{Param(parent='CountVectorizer_094638eb05f5', name='vocabSize', doc='max size of the vocabulary. Default 1 << 18.'): 1024, Param(parent='LogisticRegression_9f2332faa637', name='regParam', doc='regularization parameter (>= 0).'): 0.1}, {Param(parent='CountVectorizer_094638eb05f5', name='vocabSize', doc='max size of the vocabulary. Default 1 << 18.'): 1024, Param(parent='LogisticRegression_9f2332faa637', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='CountVectorizer_094638eb05f5', name='vocabSize', doc='max size of the vocabulary. Default 1 << 18.'): 16384, Param(parent='LogisticRegression_9f2332faa637', name='regParam', doc='regularization parameter (>= 0).'): 0.1}, {Param(parent='CountVectorizer_094638eb05f5', name='vocabSize', doc='max size of the vocabulary. Default 1 << 18.'): 16384, Param(parent='LogisticRegression_9f2332faa637', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='CountVectorizer_094638eb05f5', name='vocabSize', doc='max size of the vocabulary. Default 1 << 18.'): 262144, Param(parent='LogisticRegression_9f2332faa637', name='regParam', doc='regularization parameter (>= 0).'): 0.1}, {Param(parent='CountVectorizer_094638eb05f5', name='vocabSize', doc='max size of the vocabulary. Default 1 << 18.'): 262144, Param(parent='LogisticRegression_9f2332faa637', name='regParam', doc='regularization parameter (>= 0).'): 0.01}], Param(parent='CrossValidatorModel_698659934567', name='evaluator', doc='evaluator used to select hyper-parameters that maximize the validator metric'): BinaryClassificationEvaluator_461facc334aa}

In [0]:
print("\nThe best model was fit using parameters: ")
print("\nTokenizer: ")
print(cvModel.bestModel.stages[0].extractParamMap())

The best model was fit using parameters: 

Tokenizer: 
{Param(parent='Tokenizer_7ce745d33d54', name='outputCol', doc='output column name.'): 'words', Param(parent='Tokenizer_7ce745d33d54', name='inputCol', doc='input column name.'): 'text'}

In [0]:
print("\nCount vectorizer: ")
print(cvModel.bestModel.stages[1].extractParamMap())

Count vectorizer: 
{Param(parent='CountVectorizer_094638eb05f5', name='binary', doc='Binary toggle to control the output vector values. If True, all nonzero counts (after minTF filter applied) are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts. Default False'): False, Param(parent='CountVectorizer_094638eb05f5', name='maxDF', doc='Specifies the maximum number of different documents a term could appear in to be included in the vocabulary. A term that appears more than the threshold will be ignored. If this is an integer >= 1, this specifies the maximum number of documents the term could appear in; if this is a double in [0,1), then this specifies the maximum fraction of documents the term could appear in. Default (2^63) - 1'): 9.223372036854776e+18, Param(parent='CountVectorizer_094638eb05f5', name='minDF', doc='Specifies the minimum number of different documents a term must appear in to be included in the vocabulary. If this is an integer >= 1, this specifies the number of documents the term must appear in; if this is a double in [0,1), then this specifies the fraction of documents. Default 1.0'): 1.0, Param(parent='CountVectorizer_094638eb05f5', name='minTF', doc="Filter to ignore rare words in a document. For each document, terms with frequency/count less than the given threshold are ignored. If this is an integer >= 1, then this specifies a count (of times the term must appear in the document); if this is a double in [0,1), then this specifies a fraction (out of the document's token count). Note that the parameter is only used in transform of CountVectorizerModel and does not affect fitting. Default 1.0"): 1.0, Param(parent='CountVectorizer_094638eb05f5', name='outputCol', doc='output column name.'): 'features', Param(parent='CountVectorizer_094638eb05f5', name='vocabSize', doc='max size of the vocabulary. Default 1 << 18.'): 1024, Param(parent='CountVectorizer_094638eb05f5', name='inputCol', doc='input column name.'): 'words'}

In [0]:
print("\nLogistic regression: ")
print(cvModel.bestModel.stages[2].extractParamMap())

Logistic regression: 
{Param(parent='LogisticRegression_9f2332faa637', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_9f2332faa637', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_9f2332faa637', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_9f2332faa637', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_9f2332faa637', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_9f2332faa637', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_9f2332faa637', name='maxBlockSizeInMB', doc='maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0.'): 0.0, Param(parent='LogisticRegression_9f2332faa637', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_9f2332faa637', name='predictionCol', doc='prediction column name.'): 'prediction', Param(parent='LogisticRegression_9f2332faa637', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities.'): 'probability', Param(parent='LogisticRegression_9f2332faa637', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name.'): 'rawPrediction', Param(parent='LogisticRegression_9f2332faa637', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='LogisticRegression_9f2332faa637', name='standardization', doc='whether to standardize the training features before fitting the model.'): True, Param(parent='LogisticRegression_9f2332faa637', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].'): 0.5, Param(parent='LogisticRegression_9f2332faa637', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06}

In [0]:
# Prepare test documents, which are unlabeled.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [0]:
prediction = cvModel.transform(test)

In [0]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
print('\n')
for row in selected.collect():
    print(row)


Row(id=4, text='spark i j k', probability=DenseVector([0.2665, 0.7335]), prediction=1.0)
Row(id=5, text='l m n', probability=DenseVector([0.9204, 0.0796]), prediction=0.0)
Row(id=6, text='mapreduce spark', probability=DenseVector([0.4438, 0.5562]), prediction=1.0)
Row(id=7, text='apache hadoop', probability=DenseVector([0.8587, 0.1413]), prediction=0.0)